### nicholas chung
### data607
### 10/24/19

## assignment: web APIs

The New York Times web site provides a rich set of APIs, as described here: https://developer.nytimes.com/apis

You’ll need to start by signing up for an API key.

Your task is to choose one of the New York Times APIs, construct an interface in Python to read in the JSON data, and
transform it into a pandas DataFrame.

In [40]:
import config
import datetime
import requests
import time
import pandas as pd

In [33]:
# define how far back we want to go in our analysis 
weeks = []
# append dates for each of week of the past 3 years to list
for i in range(0, 1095, 7):
    weeks.append((datetime.date.today() - datetime.timedelta(i)).isoformat())
    
print(weeks)

# store API key as "key" for easy reference   
key = config.secret

['2019-10-24', '2019-10-17', '2019-10-10', '2019-10-03', '2019-09-26', '2019-09-19', '2019-09-12', '2019-09-05', '2019-08-29', '2019-08-22', '2019-08-15', '2019-08-08', '2019-08-01', '2019-07-25', '2019-07-18', '2019-07-11', '2019-07-04', '2019-06-27', '2019-06-20', '2019-06-13', '2019-06-06', '2019-05-30', '2019-05-23', '2019-05-16', '2019-05-09', '2019-05-02', '2019-04-25', '2019-04-18', '2019-04-11', '2019-04-04', '2019-03-28', '2019-03-21', '2019-03-14', '2019-03-07', '2019-02-28', '2019-02-21', '2019-02-14', '2019-02-07', '2019-01-31', '2019-01-24', '2019-01-17', '2019-01-10', '2019-01-03', '2018-12-27', '2018-12-20', '2018-12-13', '2018-12-06', '2018-11-29', '2018-11-22', '2018-11-15', '2018-11-08', '2018-11-01', '2018-10-25', '2018-10-18', '2018-10-11', '2018-10-04', '2018-09-27', '2018-09-20', '2018-09-13', '2018-09-06', '2018-08-30', '2018-08-23', '2018-08-16', '2018-08-09', '2018-08-02', '2018-07-26', '2018-07-19', '2018-07-12', '2018-07-05', '2018-06-28', '2018-06-21', '2018

In [3]:
# confirm authentication and successful HTTPS request
r = requests.get('https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key=' + key)
r.status_code

200

In [35]:
# store URL as fragments for ease of reuse
url_a = "https://api.nytimes.com/svc/books/v3/" 
url_b = "lists/" + d + "/hardcover-fiction.json?" 

books = []
books_d = {}

# grab values for each object returned, then store as list of dicts
for d in weeks:
    r = requests.get(url_a + url_b + "api-key=" + key)
    time.sleep(6) # rate limit guidance as per https://developer.nytimes.com/faq#a11
    for i in r.json()['results']['books']:
        # create dictionary for each result
        books_d = {
            "date": d,
            "title": i['title'], 
            "rank": i['rank'], 
            "rank_last_week": i['rank_last_week'], 
            "weeks_on_list": i['weeks_on_list'], 
            "description": i['description'], 
            "publisher": i['publisher'], 
            "author": i['author']
        }
        # append dictionary to list
        print(books_d)
        books.append(books_d)

print(books)

{'date': '2019-10-24', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-10-24', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2019-10-24', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2019-10-24', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2019-10-03', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-10-03', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2019-10-03', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2019-10-03', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2019-09-12', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-09-12', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2019-09-12', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2019-09-12', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2019-08-22', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-08-22', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2019-08-22', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2019-08-22', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2019-08-01', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-08-01', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2019-08-01', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2019-08-01', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

...

{'date': '2017-01-19', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2017-01-19', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2017-01-19', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2017-01-19', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2016-12-29', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2016-12-29', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2016-12-29', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2016-12-29', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2016-12-08', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2016-12-08', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2016-12-08', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2016-12-08', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2016-11-17', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2016-11-17', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2016-11-17', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2016-11-17', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

{'date': '2016-10-27', 'title': 'THE OUTSIDER', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A detective investigates a seemingly wholesome member of the community when an 11-year-old boy’s body is found.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2016-10-27', 'title': 'SHELTER IN PLACE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Survivors of a mass shooting outside a mall in Portland, Me., develop different coping mechanisms and face a new menace.', 'publisher': "St. Martin's", 'author': 'Nora Roberts'}
{'date': '2016-10-27', 'title': 'THE GRAY GHOST', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Sam and Remi Fargo search for a Rolls-Royce prototype that has vanished for a second time.', 'publisher': 'Putnam', 'author': 'Clive Cussler and Robin Burcell'}
{'date': '2016-10-27', 'title': 'THE DEATH OF MRS. WESTAWAY', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A tarot card

In [41]:
# store list of dictionanies to dataframe
books_df = pd.DataFrame(books)
books_df

,date,title,rank,rank_last_week,weeks_on_list,description,publisher,author
0,2019-10-24,THE OUTSIDER,1,1,2,A detective investigates a seemingly wholesome...,Scribner,Stephen King
1,2019-10-24,SHELTER IN PLACE,2,0,1,Survivors of a mass shooting outside a mall in...,St. Martin's,Nora Roberts
2,2019-10-24,THE GRAY GHOST,3,0,1,Sam and Remi Fargo search for a Rolls-Royce pr...,Putnam,Clive Cussler and Robin Burcell
3,2019-10-24,THE DEATH OF MRS. WESTAWAY,4,0,1,A tarot card reader mistakenly receives an inh...,Gallery/Scout,Ruth Ware
4,2019-10-24,TO THE MOON AND BACK,5,0,1,A man whose mother died in the Oklahoma City b...,Howard,Karen Kingsbury
...,...,...,...,...,...,...,...,...
2350,2016-10-27,LITTLE FIRES EVERYWHERE,11,11,37,An artist upends a quiet town outside Cleveland.,Penguin Press,Celeste Ng
2351,2016-10-27,THE HIGH TIDE CLUB,12,9,4,An eccentric millionaire enlists the attorney ...,St. Martin's,Mary Kay Andrews
2352,2016-10-27,THE WOMAN IN THE WINDOW,13,13,22,A recluse who drinks heavily and takes prescri...,Morrow,AJ Finn
2353,2016-10-27,THE GREAT ALONE,14,12,17,A former prisoner of war moves his family to A...,St. Martin's,Kristin Hannah
